**<font color = white size=6 >如何將資料丟到CUDA。</font>**


在torch的tensor下直接.to(device)即可，但device需要先宣告。

In [ ]:
# 確認GPU數量

In [2]:
from torchvision import datasets, transforms
import torch
transform = transforms.ToTensor()
dataset_MNIST_tensor = datasets.MNIST('../data/cv', train=True, download=True, transform=transform)
mnistdata_loader = torch.utils.data.DataLoader(dataset_MNIST_tensor, batch_size=2)


use_cuda = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() & use_cuda) else "cpu")
print(device)

for data, target in mnistdata_loader:
    print(target)
    data, target = data.to(device), target.to(device)
    print(target)
    break

cuda:0
tensor([5, 0])
tensor([5, 0], device='cuda:0')


## 我們針對CPU mode的tensor和CUDA mode的tensor做操作看看

### 1. CPU Tensor 相加

In [4]:
use_cuda = 0
device = torch.device("cuda" if use_cuda else "cpu")

one = torch.Tensor([1])
for data, target in mnistdata_loader:
    print(data+one)
    break

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]])


### 2. CUDA Tensor 加 CPU Tensor

In [5]:
one = torch.Tensor([1])
for data, target in mnistdata_loader:
    data, target = data.to(device), target.to(device)
    print(data)
    print(data+one)
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
     

### 3. CUDA Tensor 加 CUDA Tensor

In [7]:
use_cuda = 1
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

one = torch.Tensor([1])
one = one.to(device)
for data, target in mnistdata_loader:
    data, target = data.to(device), target.to(device)
    print(data+one)
    break

cuda:0
tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], device='cuda:0')


### 4. 測試CPU和CUDA之間資料搬移

In [8]:
import time
def test_time(data_loader):
    start = time.time()
    count=0
    for data, target in data_loader:
        count+=1
    print("a forloop time for whole dataset within CPU: {}s".format(time.time()-start))

    start = time.time()
    count=0
    for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        count+=1
    print("a forloop time for whole dataset with CPU to CUDA: {}s".format(time.time()-start))
    
print("When batch size = 10")
mnistdata_loader = torch.utils.data.DataLoader(dataset_MNIST_tensor, batch_size=10, shuffle=False)
test_time(mnistdata_loader)

print("\nWhen batch size = 2")
mnistdata_loader = torch.utils.data.DataLoader(dataset_MNIST_tensor, batch_size=2, shuffle=False)
test_time(mnistdata_loader)


When batch size = 10
a forloop time for whole dataset within CPU: 2.253023862838745s
a forloop time for whole dataset with CPU to CUDA: 4.19034218788147s

When batch size = 2
a forloop time for whole dataset within CPU: 2.874025583267212s
a forloop time for whole dataset with CPU to CUDA: 8.376002311706543s


</font>**<font color = black size=4 >Note: </font>**<br>

**<font color = black size=3 >1:資料跑來跑去一定花費data bandwidth，導致時間會變慢，所以在pytorch撰寫過程中要盡量避免資料在CPU和GPU之間跑來跑去。</font>**<br>

**<font color = black size=3 >2: 容易造成CPU的tensor和CUDA的tensor進行運算的error，在進行運算要注意是在CPU還是CUDA。</font>**
